# Day 8
## Part 1

In [1]:
type Direction = | Left | Right

let parseDirections (s: string) : Direction list =
    s 
    |> Seq.map (function
        | 'L' -> Left
        | 'R' -> Right
        | _ -> failwith "Invalid direction"
    ) |> Seq.toList

type Node = { LeftName : string; RightName : string }

let parseNode (s: string) (nodes: Map<string, Node>) =
    let parts = s.Split(Seq.toArray " =(),", StringSplitOptions.RemoveEmptyEntries)
    nodes
    |> Map.add parts[0] { LeftName = parts[1]; RightName = parts[2] }

let parseInput parseDirections parseNode inputLines =
    let directions = parseDirections (Array.head inputLines)
    let nodes =
        inputLines
        |> Array.skip 2
        |> Array.fold (fun nodes line -> parseNode line nodes) Map.empty
    (directions, nodes)

let followDirection nodes currentNodeName direction =
    let node = nodes |> Map.find currentNodeName
    match direction with
    | Left -> node.LeftName
    | Right -> node.RightName

let repeat xs = 
    seq {
        while true do yield! xs
    }

let findEndpoint nodes startName endName directions =
    repeat directions
    |> Seq.scan (followDirection nodes) startName
    |> Seq.takeWhile (fun name -> name <> endName)

let countSteps findEnd =
    findEnd
    |> Seq.length // Not + 1 for the end position because this includes the starting position


In [2]:
#r "nuget: FsUnit.xUnit"

open FsUnitTyped

let testData = 
    [|
        "LLR"
        ""
        "AAA = (BBB, BBB)"
        "BBB = (AAA, ZZZ)"
        "ZZZ = (ZZZ, ZZZ)"
    |]

let testResult =
    let (directions, nodes) = parseInput parseDirections parseNode testData
    findEndpoint nodes "AAA" "ZZZ" directions
    |> countSteps

testResult |> shouldEqual 6

Installed Packages FsUnit.xUnit, 5.6.1

In [3]:
open System.IO
let input = File.ReadAllLines "input_08.txt"

let result1 =
    let (directions, nodes) = parseInput parseDirections parseNode input
    findEndpoint nodes "AAA" "ZZZ" directions
    |> countSteps

printfn "Result 1 : %d" result1

result1 |> shouldEqual 12169 // In case I break it

Result 1 : 12169


## Part 2

In [8]:
let findEndpoint2 nodes startName isEndName directions =
    repeat directions
    |> Seq.scan (followDirection nodes) startName
    |> Seq.takeWhile (fun name -> not (isEndName name))    

let startingNodes nodes =
    nodes 
    |> Map.keys 
    |> Seq.filter (fun (name : string) -> name.EndsWith("A"))
    |> List.ofSeq

let endsWithZ (name : string) = name.EndsWith("Z")

let stepsPerRoute directions nodes starts =
    starts
    |> List.map (
        fun start -> 
            findEndpoint2 nodes start endsWithZ directions
            |> countSteps
    )

#r "nuget: MathNet.Numerics"

let lcm (xs: int list) =
    MathNet.Numerics.Euclid.LeastCommonMultiple (xs |> Seq.map bigint |> Seq.toArray)


Installed Packages MathNet.Numerics, 5.0.0

In [9]:
lcm [1; 2; 3] |> shouldEqual 6I

let testInput2 = 
    [|
        "LR"
        ""
        "11A = (11B, XXX)"
        "11B = (XXX, 11Z)"
        "11Z = (11B, XXX)"
        "22A = (22B, XXX)"
        "22B = (22C, 22C)"
        "22C = (22Z, 22Z)"
        "22Z = (22B, 22B)"
        "XXX = (XXX, XXX)"
    |]

let testResult2 =
    let (directions, nodes) = parseInput parseDirections parseNode testInput2
    startingNodes nodes
    |> stepsPerRoute directions nodes
    |> lcm

testResult2 |> shouldEqual 6I

In [11]:
let result2 = 
    let (directions, nodes) = parseInput parseDirections parseNode input
    startingNodes nodes
    |> stepsPerRoute directions nodes
    |> lcm

printfn "Result 2: %A" result2 // 5 minutes, could use a library for lcm
result2 |> shouldEqual 12030780859469I // In case I break it

Result 2: 12030780859469
